# CSCI 6907: Computational Photography | Project 01
## Aligning and compositing the images of the Prokudin-Gorskii photo collection
### Team Members: Charles Peeke, Shivam Shah

In [ ]:
import numpy as np
import skimage as sk
import skimage.io as skio
import cv2

from skimage.transform import rescale

In [ ]:
name='camel'

# name of the input file
imname = 'data/'+name+'.jpg'

# read in the image
im = skio.imread(imname)

# convert to double (might want to do this later on to save memory)    
im = sk.img_as_float(im)
    
# compute the height of each part (just 1/3 of total)
height = int(np.floor(im.shape[0] / 3.0))

# separate color channels
b = im[:height]
g = im[height: 2*height]
r = im[2*height: 3*height]



In [ ]:
# create a color image
im_out = np.dstack([r, g, b])

# save the image
fname = 'results/out_'+name+'base.jpg'
skio.imsave(fname, im_out)

# display the image
skio.imshow(im_out)
skio.show()

In [ ]:
def align_force(img, base_img):
    best_score = np.sum((base_img - img)**2)
    adj_image = 0
    for i in range(-15, 15):
        for j in range(-15, 15):
            rollx = np.roll(img, i, axis=0)
            test_img = np.roll(rollx, j, axis=1)
            
            test_score = np.sum((base_img - test_img)**2)
            
            if (test_score < best_score):
                adj_image = test_img
                best_score = test_score
                offsetx = i
                offsety = j
    return adj_image

In [ ]:
%%time
ag = align_force(g, b)
ar = align_force(r, b)

In [ ]:
# create a color image
im_out_force = np.dstack([ar, ag, b])

# save the image
fname = 'results/out_'+name+'_force.jpg'
test = fname
skio.imsave(fname, im_out_force)

# display the image
skio.imshow(im_out_force)
skio.show()

In [ ]:
def align_force_third(img, base_img):
    thirdx = int(base_img.shape[0]/3)
    thirdy = int(base_img.shape[1]/3)
#     [thirdx:thirdx*2, thirdy:thirdy*2]

    best_score = np.sum((base_img[thirdx:thirdx*2, thirdy:thirdy*2] - img[thirdx:thirdx*2, thirdy:thirdy*2])**2)
    adj_image = 0
    
    for i in range(-15, 15):
        for j in range(-15, 15):
            rollx = np.roll(img, i, axis=0)
            test_img = np.roll(rollx, j, axis=1)
            
            test_score = np.sum((base_img[thirdx:thirdx*2, thirdy:thirdy*2] - test_img[thirdx:thirdx*2, thirdy:thirdy*2])**2)
            
            if (test_score < best_score):
                adj_image = test_img
                best_score = test_score
                offsetx = i
                offsety = j

    return adj_image

In [ ]:
%%time
ag2 = align_force_third(g, b)
ar2 = align_force_third(r, b)

In [ ]:
# create a color image
im_out_force2 = np.dstack([ar2, ag2, b])

# save the image
fname = 'results/out_'+name+'_force2.jpg'
test = fname
skio.imsave(fname, (im_out_force2*255).astype(np.uint8))

# display the image
skio.imshow(im_out_force2)
skio.show()

In [ ]:
def align_force_basiccropping(img, base_img):    
    
    img_cropped = img[int(0.1 * len(img)):-int(0.1 * len(img)), 
                      int(0.1 * len(img[0])):-int(0.1 * len(img[0]))]
    base_img_cropped = base_img[int(0.1 * len(base_img)):-int(0.1 * len(base_img)), 
                                int(0.1 * len(base_img[0])):-int(0.1 * len(base_img[0]))]
    
    best_score = np.sum((base_img_cropped - img_cropped)**2)

    adj_image = 0
    
    for i in range(-15, 15):
        for j in range(-15, 15):
            test_img_cropped = np.roll(np.roll(img_cropped, i, axis=0), j, axis=1)
            
            test_score = np.sum((base_img_cropped - test_img_cropped)**2)
            
            if (test_score < best_score):
                adj_image = np.roll(img, [i,j], (0,1)), np.array([i,j])
                best_score = test_score
                offsetx = i
                offsety = j

#     print(best_score, offsetx, offsety)
    return adj_image[0]


In [ ]:
%%time
ag2 = align_force_basiccropping(g, b)
ar2 = align_force_basiccropping(r, b)

In [ ]:
# create a color image
im_out_force2 = np.dstack([ar2, ag2, b])

# save the image
fname = 'results/out_'+name+'_force_Cropped.jpg'
test = fname
skio.imsave(fname, (im_out_force2*255).astype(np.uint8))

# display the image
skio.imshow(im_out_force2)
skio.show()

In [ ]:
RGB_SCALE = 255
CMYK_SCALE = 100


def rgb_to_cmyk(r, g, b):
    if (r, g, b) == (0, 0, 0):
        # black
        return 0, 0, 0, CMYK_SCALE

    # rgb [0,255] -> cmy [0,1]
    c = 1 - r / RGB_SCALE
    m = 1 - g / RGB_SCALE
    y = 1 - b / RGB_SCALE

    # extract out k [0, 1]
    min_cmy = min(c, m, y)
    c = (c - min_cmy) / (1 - min_cmy)
    m = (m - min_cmy) / (1 - min_cmy)
    y = (y - min_cmy) / (1 - min_cmy)
    k = min_cmy

    # rescale to the range [0,CMYK_SCALE]
    return c * CMYK_SCALE, m * CMYK_SCALE, y * CMYK_SCALE, k * CMYK_SCALE





def better_colors(img):
    from scipy.special import logsumexp
    rows,cols = (img.shape[0], img.shape[1])
    
    clist = [[0]*cols]*rows
    mlist = [[0]*cols]*rows
    ylist = [[0]*cols]*rows
    klist = [[0]*cols]*rows
    
   #print(rows)

    
    for rr in range(img.shape[0]):
        for c in range(img.shape[0]):
            
            C,M,Y,K = rgb_to_cmyk(img[rr,c,2],img[rr,c,1],img[rr,c,0])
            
            klist[rr][c] = K
            clist[rr][c] = C
            mlist[rr][c] = M
            ylist[rr][c] = Y
            
            
    karr = np.array(klist)
    carr = np.array(clist)
    marr = np.array(mlist)
    yarr = np.array(ylist)



    im_out_1 = np.dstack([carr, marr, yarr, karr])
    im_out_2 = np.dstack([karr, marr, yarr, carr])
    im_out_3 = np.dstack([karr, carr, yarr, marr])
    im_out_4 = np.dstack([karr, marr, carr, yarr])
    return [im_out_1, im_out_2, im_out_3, im_out_4]

In [ ]:
'''
cmyk = better_colors(im_out)

im_out_1 = cmyk[0]

print(im_out_1.shape)
fname = 'results/out_'+name+'_cmyk1.jpg'
skio.imsave(fname, im_out_1, )
skio.imshow(im_out_1)
skio.show()
'''

In [ ]:
%%time
from PIL import Image
image = Image.open(test)


fname = 'results/out_'+name+'_cmyk2.jpg'
if image.mode == 'RGB':
    cmyk_image = image.convert('CMYK')
cmyk_image.save(fname)
# cmyk_image.show()

In [ ]:
def align_force_edges(img, base_img):

    img2 = (img*255).astype(np.uint8)
    base_img = (base_img*255).astype(np.uint8)

    img_edges = cv2.Canny(img2, 255/3, 255)
    base_img_edges = cv2.Canny(base_img, 255/3, 255)


    best_score = np.sum((base_img_edges - img_edges)**2)

    adj_image = 0
    
    # Shifting the image
    for i in range(-15, 15):
        for j in range(-15, 15):
            rollx = np.roll(img2, i, axis=0)
            rolly = np.roll(rollx, j, axis=1)

            test_img = cv2.Canny(rolly, 255/3, 255)
            
            test_score = np.sum((base_img_edges - test_img)**2)
            
            if (test_score < best_score):
                adj_image = np.roll(img, [i,j], (0,1)), np.array([i,j])
                best_score = test_score

    return adj_image[0]

In [ ]:
%%time
ag_edges = align_force_edges(g, b)
ar_edges = align_force_edges(r, b)


In [ ]:

r_edges = cv2.Canny((r*255).astype(np.uint8), 255/5, 255)
g_edges = cv2.Canny((g*255).astype(np.uint8), 255/5, 255)
b_edges = cv2.Canny((b*255).astype(np.uint8), 255/5, 255)

# create a color image
im_out = np.dstack([ar_edges, ag_edges, b])

# save the image
fname = 'results/out_'+name+'_edges.jpg'
fname_r = 'results/out_'+name+'_red_edges.jpg'
fname_g = 'results/out_'+name+'_green_edges.jpg'
fname_b = 'results/out_'+name+'_blue_edges.jpg'

skio.imsave(fname, sk.util.img_as_ubyte(im_out))
skio.imsave(fname_r, sk.util.img_as_ubyte(r_edges))
skio.imsave(fname_g, sk.util.img_as_ubyte(g_edges))
skio.imsave(fname_b, sk.util.img_as_ubyte(b_edges))


# display the image
# skio.imshow(im_out)
# skio.show()


In [ ]:
#Another Approch using an image pyramid

# align the images
# functions that might be useful for aligning the images include:
# np.roll, np.sum, sk.transform.rescale (for multiscale) # Downscaling
def align_force_pyramid (img, base_img,off_x=(-15,15),off_y=(-15,15)):
    # Not needed - but focuses the differences based on the middle of the image
    thirdx = int(base_img.shape[0]/3)
    thirdy = int(base_img.shape[1]/3)
#     [thirdx:thirdx*2, thirdy:thirdy*2]
    # Gather base scores for unshifted image
    best_score = np.sum((base_img[thirdx:thirdx*2, thirdy:thirdy*2] - img[thirdx:thirdx*2, thirdy:thirdy*2])**2)
#     print('original', best_score)
    adj_image = 0
    offsetx = 0
    offsety = 0
    
    # Shifting the image
    
   # print(type(off_x))
    temp_x = np.array(off_x)
    temp_y = np.array(off_y)
    
    for i in range(temp_x[0], temp_x[1]):
        for j in range(temp_y[0], temp_y[1]):
            rollx = np.roll(img, i, axis=0)
            test_img = np.roll(rollx, j, axis=1)
            
            test_score = np.sum((base_img[thirdx:thirdx*2, thirdy:thirdy*2] - test_img[thirdx:thirdx*2, thirdy:thirdy*2])**2)
            
            if (test_score < best_score):
                adj_image = test_img
                best_score = test_score
                offsetx = i
                offsety = j
            

    #print(offsetx, offsety)
    return [offsetx,offsety]


In [ ]:
def pyramid (im1 , im2 , offx=(-4,4) , offy=(-4,4) , depth=5):
    
    if depth == 0 or im1.shape[0] < 400:
        test = align_force_pyramid(im1,im2)
        #print(test)
        return (test)
    else:
        best_shift = pyramid(rescale(im1, 0.5), rescale(im2, 0.5), depth=depth - 1)
        #print(best_shift)
       # print(im1.shape)
        #print(im2.shape)

        # scale the displacement shift vector back up
        
        
        best_shift = [x*2 for x in best_shift]
        
        #print(best_shift)
     
        # find the best shifted image and displacement vector at this level
        new_shift = align_force_pyramid(np.roll(im1, best_shift, (0, 1)), im2, offx , offy)
        
        # add the new displacement shift vector to the best_shift vector and return
        best_shift = [a+b for a,b in zip(best_shift,new_shift)]
       #print(best_shift)
        
        return best_shift

In [ ]:
g_new = pyramid(g,b,5)

#print(g_new)
#print(g)

#print(g_new)

r_new = pyramid(r,b,5)

pg = np.roll(g,g_new,(0,1))



pr = np.roll(r,r_new,(0,1))

#print(pr)

im_out_pyr = np.dstack([pr,pg,b])

print(im_out_pyr.shape)

fname = 'results/out_'+name+'_pyramid.jpg'
skio.imsave(fname, im_out_pyr)

# display the image
skio.imshow(im_out_pyr)
skio.show()



In [ ]:
from PIL import Image
def autocrop(im):

    non_empty_columns = np.where(im.max(axis=0)>0)[0]
    non_empty_rows = np.where(im.max(axis=1)>0)[0]
    cropBox = (min(non_empty_rows), max(non_empty_rows), min(non_empty_columns), max(non_empty_columns))

    image_data_new = im[cropBox[0]:cropBox[1]+1, cropBox[2]:cropBox[3]+1]
    return image_data_new
    

In [ ]:
im = cv2.imread('data/objects.jpg')
imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
imgray = cv2.blur(imgray,(15,15))
ret,thresh = cv2.threshold(imgray,int(np.average(imgray)),255,cv2.THRESH_BINARY_INV)
dilated=cv2.morphologyEx(thresh, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10)))
contours = cv2.findContours(dilated,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
new_contours=[]
siz = (im.shape[0]*im.shape[1])*0.001
for c in contours:
    if cv2.contourArea(c)< 44:
        new_contours.append(c)
print(len(new_contours))
best_box=[-1,-1,-1,-1]
for c in new_contours[:]:
   x,y,w,h = cv2.boundingRect(c)
   if best_box[0] < 0:
       best_box=[x,y,x+w,y+h]
   else:
       if x<best_box[0]:
           best_box[0]=x
       if y<best_box[1]:
           best_box[1]=y
       if x+w>best_box[2]:
           best_box[2]=x+w
       if y+h>best_box[3]:
           best_box[3]=y+h

In [ ]:
im_out_box = cv2.rectangle(im, (best_box[0], best_box[1]), (best_box[2], best_box[3]), (255,0,0), 30)
skio.imsave('results/out_objects_boundingBox.jpg', im_out_box)

In [ ]:
skio.imshow(im_out_box)
skio.show()

In [ ]:
img = cv2.imread('data/'+name+'.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
_,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
skio.imshow(thresh)
skio.show()

contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
# contours = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
cnt = contours[0]
x,y,w,h = cv2.boundingRect(cnt)
crop = img[y:y+h, x:x+w]
print(x, y, w, h)
print(img.shape)
cv2.imwrite('results/out_'+name+'_SIMPLEAUTO.jpg',crop)
